In [11]:
!wget -P "../data/"  https://github.com/mattharrison/datasets/raw/master/data/__mharrison__2020-2021.csv

--2024-01-14 12:48:00--  https://github.com/mattharrison/datasets/raw/master/data/__mharrison__2020-2021.csv
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mattharrison/datasets/master/data/__mharrison__2020-2021.csv [following]
--2024-01-14 12:48:01--  https://raw.githubusercontent.com/mattharrison/datasets/master/data/__mharrison__2020-2021.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1725101 (1.6M) [text/plain]
Saving to: ‘../data/__mharrison__2020-2021.csv’

__mharrison__2020-2 100%[===================>]   1.64M  5.50MB/s    in 0.3s    

2024-01-14 12:48:02 (5.50 MB/s) - ‘../data/__m

In [12]:
!mv ../data/__mharrison__2020-2021.csv ../data/mharrison__2020-2021.csv

# Ok I understand, this line needs some explanation. Apparently, PySpark can't automatically inferSchema for CSV files which start with an underscore. Why can't it read files with underscores, you ask? I have no clue, Perhaps the shamans at Databricks have a longstanding fued with the UnderScore Lords. I merely ride the waves(and spend half an hour wondering if I was a moron before stumbling onto this answer(https://stackoverflow.com/a/73456507/14274188)).

In [1]:
# Internal

# External
from pyspark.sql import SparkSession

# Project

# Project Specific Configuration

sc = SparkSession.builder.getOrCreate()

24/01/14 13:40:54 WARN Utils: Your hostname, codespaces-861db6 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/01/14 13:40:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/14 13:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = sc.read.option("header","true").csv("../data/mharrison__2020-2021.csv")

In [3]:
df.count()

7076

In [4]:
df.show()

24/01/14 13:41:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+--------------------+--------------------+--------------------+-----------+-----------+------------------+--------+-------+-----+-------------------+----------+--------------+--------------+----------------+---------+------------+-------+-----------+----------+-----------+-----------------+--------------------+--------------------+------------------------+-----------------+----------------+--------------+----------------------------+-------------------+-----------------------+-----------------------+-------------------------+------------------+---------------------+----------------+--------------------+-------------------+--------------------+--------------------------+
|           Tweet id|     Tweet permalink|          Tweet text|                time|impressions|engagements|   engagement rate|retweets|replies|likes|user profile clicks|url clicks|hashtag clicks|detail expands|permalink clicks|app opens|app installs|follows|email tweet|dial phone|media views|media 

In [5]:
dir(df)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_ipython_key_completions_',
 '_jcols',
 '_jdf',
 '_jmap',
 '_joinAsOf',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_session',
 '_sort_cols',
 '_sql_ctx',
 '_support_repr_html',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'dropDuplicatesWithinWatermark',
 'drop_du

In [6]:
df.columns

['Tweet id',
 'Tweet permalink',
 'Tweet text',
 'time',
 'impressions',
 'engagements',
 'engagement rate',
 'retweets',
 'replies',
 'likes',
 'user profile clicks',
 'url clicks',
 'hashtag clicks',
 'detail expands',
 'permalink clicks',
 'app opens',
 'app installs',
 'follows',
 'email tweet',
 'dial phone',
 'media views',
 'media engagements',
 'promoted impressions',
 'promoted engagements',
 'promoted engagement rate',
 'promoted retweets',
 'promoted replies',
 'promoted likes',
 'promoted user profile clicks',
 'promoted url clicks',
 'promoted hashtag clicks',
 'promoted detail expands',
 'promoted permalink clicks',
 'promoted app opens',
 'promoted app installs',
 'promoted follows',
 'promoted email tweet',
 'promoted dial phone',
 'promoted media views',
 'promoted media engagements']

24/01/14 13:41:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Transpose is not very straightforward with the stand `sparkDataFrame`. There's a complex way to do it [here](https://stackoverflow.com/questions/71474821/how-i-can-transpose-a-data-frame-in-pyspark) using pivots and some voodoo magic, but for most exploratory purposes we can simple switch to the [PandasAPI on Spark](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html) and use its transpose tool.

In [8]:


df.sample(0.1).pandas_api().T

0                                                             1                                                                                     2                                                             3                                                             4                                                             5                                                                                 6                                                             7                                                             8                                                             9                                                             10                                                            11                                                                                                                                                           12                                                                                                                                                                       13                                                            14                                                            15                                                            16                                                                                      17                                                                                      18                                                            19                                                                                                                   20                                                            21                                                                                    22                                                                                                                    23                                                                                                      24                                                                                                             25                                                              26                      27                                       28                                                                   29                                                            30                                                                   31                                                                    32                                                            33                                                                                   34                                                                                                                                 35                                                                                                                                                                                                                                                               36                                                            37                                                            38                                                            39                                                            40                                                            41                                                            42                                                                                                                                                                 43                                                               44                                                            45                                                                                                     46                                                            47                                                                            48                                                                   49                

In [10]:
df.count(),len(df.columns)

(7076, 40)

In [11]:
df.dtypes

[('Tweet id', 'string'),
 ('Tweet permalink', 'string'),
 ('Tweet text', 'string'),
 ('time', 'string'),
 ('impressions', 'string'),
 ('engagements', 'string'),
 ('engagement rate', 'string'),
 ('retweets', 'string'),
 ('replies', 'string'),
 ('likes', 'string'),
 ('user profile clicks', 'string'),
 ('url clicks', 'string'),
 ('hashtag clicks', 'string'),
 ('detail expands', 'string'),
 ('permalink clicks', 'string'),
 ('app opens', 'string'),
 ('app installs', 'string'),
 ('follows', 'string'),
 ('email tweet', 'string'),
 ('dial phone', 'string'),
 ('media views', 'string'),
 ('media engagements', 'string'),
 ('promoted impressions', 'string'),
 ('promoted engagements', 'string'),
 ('promoted engagement rate', 'string'),
 ('promoted retweets', 'string'),
 ('promoted replies', 'string'),
 ('promoted likes', 'string'),
 ('promoted user profile clicks', 'string'),
 ('promoted url clicks', 'string'),
 ('promoted hashtag clicks', 'string'),
 ('promoted detail expands', 'string'),
 ('promo